In [33]:
import random
import numpy as np
import pandas as pd
import joblib as jlb
from PIL import Image
from collections import OrderedDict
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from os import listdir

data_train = pd.read_csv('./data/class_data.csv', header=None).to_numpy()
data_test = pd.read_csv('./data/class_test.csv', header=None).to_numpy()
label = pd.read_csv('./data/training-label.csv', header=None).to_numpy()

print(data_train.shape)
print(data_test.shape)
print(label.shape)

mac_hash_train = data_train[1:, 1]
sniff_train = data_train[1:, 2]
time_train = data_train[1:, 3]
class_train = data_train[1:, 4]
dist_train = data_train[1:, 5]
print(len(dist_train))

mac_hash_test = data_test[1:, 1]
sniff_test = data_test[1:, 2]
time_test = data_test[1:, 3]
class_test = data_test[1:, 4]
dist_test = data_test[1:, 5]
print(len(mac_hash_test))

label = label[1:, :]

time_list_train = list(time_train)
d_train = {'Datatime': time_list_train}
df_train = pd.DataFrame(data=d_train)
df_train = pd.to_datetime(df_train['Datatime'], format = "%Y-%m-%d %H:%M:%S")

time_list_test = list(time_test)
d_test = {'Datatime': time_list_test}
df_test = pd.DataFrame(data=d_test)
df_test = pd.to_datetime(df_test['Datatime'], format = "%Y-%m-%d %H:%M:%S")

(41641, 6)
(20420, 6)
(6960, 2)
41640
20419


In [34]:
time_train = []
for i in range(len(mac_hash_train)):
    time_train.append(float(df_train[i].hour*3600+df_train[i].minute*60+df_train[i].second))

time_test = []
for i in range(len(mac_hash_test)):
    time_test.append(float(df_test[i].hour*3600+df_test[i].minute*60+df_test[i].second))

time_train = np.array(time_train)
time_test = np.array(time_test)

print(time_train)
print(time_test)

[60488. 59375. 59441. ... 67673. 67674. 67674.]
[62460. 49500. 49500. ... 54180. 54840. 55560.]


In [35]:
dist_train = np.array(dist_train)
dist_test = np.array(dist_test)
dist_scaler = StandardScaler().fit(dist_train.reshape(-1, 1))

dist_train = dist_scaler.transform(dist_train.reshape(-1, 1))
dist_train = np.ndarray.flatten(dist_train)

dist_test = dist_scaler.transform(dist_test.reshape(-1, 1))
dist_test = np.ndarray.flatten(dist_test)

print(dist_train)
print(dist_test)

[-1.32159609  0.12620552 -0.55959524 ... -0.15845227 -0.51157807
 -1.32159609]
[-1.32159609 -0.78819549 -0.51157807 ...  0.86044227  0.58340603
 -1.32159609]


In [36]:
mh_list_train = list(OrderedDict.fromkeys(mac_hash_train))

l_train = []
for i in range(len(mh_list_train)):
    l_train.insert(i, [[mh_list_train[i]]])
    
for i in range(len(mac_hash_train)):
    ll = [float(sniff_train[i]), float(df_train[i].day-6), time_train[i], dist_train[i], float(class_train[i])]
#     ll = [float(sniff_train[i]), float(df_train[i].day-6), time_train[i]]
    l_train[mh_list_train.index(mac_hash_train[i])].append(ll)

num = 0
for i in range(len(l_train)):
    num = num + len(l_train[i]) - 1

print(l_train[200])
print(len(l_train))
print(num)

[['07395cfb077764337994f6598a1336b0'], [5.0, 2.0, 57409.0, -1.3215960856502083, 2.0]]
6959
41640


In [37]:
mh_list_test = list(OrderedDict.fromkeys(mac_hash_test))

l_test = []
for i in range(len(mh_list_test)):
    l_test.insert(i, [[mh_list_test[i]]])
    
for i in range(len(mac_hash_test)):
    ll = [float(sniff_test[i]), float(df_test[i].day-6), time_test[i], dist_test[i], float(class_test[i])]
#     ll = [float(sniff_test[i]), float(df_test[i].day-6), time_test[i]]
    l_test[mh_list_test.index(mac_hash_test[i])].append(ll)

num = 0
for i in range(len(l_test)):
    num = num + len(l_test[i]) - 1

print(len(l_test))
print(num)

3429
20419


In [38]:
d_train = np.zeros((len(l_train), 14))
d_test = np.zeros((len(l_test), 14))
l_diff = []

############### train ###################
for i in range(len(l_train)):
    for j in range(1, len(l_train[i])-1):
        if l_train[i][j][1] == l_train[i][j+1][1]:
            diff = l_train[i][j+1][2] - l_train[i][j][2]
            if diff < 0:
                temp = l_train[i][j+1]
                l_train[i][j+1] = l_train[i][j]
                l_train[i][j] = temp
                diff *= -1
            d_train[i][int(l_train[i][j][0])-1] = diff
            l_diff.append(diff)


###############  test  ###################
for i in range(len(l_test)):
    for j in range(1, len(l_test[i])-1):
        if l_test[i][j][1] == l_test[i][j+1][1]:
            diff = l_test[i][j+1][2] - l_test[i][j][2]
            if diff < 0:
                temp = l_test[i][j+1]
                l_test[i][j+1] = l_test[i][j]
                l_test[i][j] = temp
                diff *= -1
            d_test[i][int(l_test[i][j][0])-1] = diff

print(len(l_diff))

34367


In [39]:
time_diff = np.array(l_diff)

print('mean', np.mean(time_diff))
# print('80', np.percentile(time_diff, 80))
# print('60', np.percentile(time_diff, 60))
# print('40', np.percentile(time_diff, 40))
# print('20', np.percentile(time_diff, 20))

data = time_diff
n = 1.5
#IQR = Q3-Q1
IQR = np.percentile(data, 75) - np.percentile(data, 25)
print('IQR', IQR)

mid = np.percentile(time_diff, 50)
mean = np.mean(time_diff)
max_limit = np.percentile(time_diff, 75)
min_limit = np.percentile(time_diff, 25)
if min_limit < 30:
    min_limit = 30.

print('mid', mid)
print('max', max_limit)
print('min', min_limit)

mean 441.77137952105215
IQR 301.0
mid 56.0
max 302.0
min 30.0


In [40]:
label_list = list(label[:, 1])

l2 = []
for i in range(len(mh_list_train)):
    l2.insert(i, [mh_list_train[i]])
    
for i in range(len(label)):
    target = int(label_list[i])
    for j in range(5):
        if target == j:
            k = 1.
        else:
            k = 0.
        l2[mh_list_train.index(label[i][0])].append(k)

print(len(l2))
dict2 = {}
for i in range(len(label)):
    dict2[i] = l2[i]

# print(dict2)
jlb.dump(dict2, 'train_label.pkl')

# print(dict2)

6959


['train_label.pkl']

In [41]:
dict2

{0: ['0003ae8541d0e925fcee242287e2ad27', 1.0, 0.0, 0.0, 0.0, 0.0],
 1: ['00078611037990f7f36b722f22595fe7', 0.0, 1.0, 0.0, 0.0, 0.0],
 2: ['000ce31739b333d931813de403156844', 0.0, 0.0, 0.0, 1.0, 0.0],
 3: ['0027240b2aeb1f7f3c053c073cafb452', 0.0, 0.0, 0.0, 0.0, 1.0],
 4: ['0028a89964416dc9a8a66cc7fe25ef4e', 0.0, 0.0, 0.0, 0.0, 1.0],
 5: ['002ccce0dc6384e91e6aaa988b542c45', 1.0, 0.0, 0.0, 0.0, 0.0],
 6: ['00414318025c30f1360225ad736aa298', 0.0, 0.0, 0.0, 0.0, 1.0],
 7: ['0041999224addbf0753762647ac2166b', 0.0, 0.0, 0.0, 0.0, 1.0],
 8: ['0044e62541626e25e041b4baf3daea4c', 0.0, 0.0, 0.0, 0.0, 1.0],
 9: ['004c76bfe9f7b20f3651e0bcbd8eee9e', 0.0, 1.0, 0.0, 0.0, 0.0],
 10: ['0052939102fb0acda914e30b560a5900', 0.0, 1.0, 0.0, 0.0, 0.0],
 11: ['00605a68f5fe01ee37e7fb6169fdbc5c', 0.0, 0.0, 1.0, 0.0, 0.0],
 12: ['007776e9906aba0186587d85895a472a', 1.0, 0.0, 0.0, 0.0, 0.0],
 13: ['00782214b7ce1c28fd9d2a9347c0520f', 1.0, 0.0, 0.0, 0.0, 0.0],
 14: ['0084303d531091816387956ea26b86eb', 0.0, 1.0, 0.0, 0

In [42]:
l_location = []
for i in range(1,15):
    l_location.append(i)

label_mac = list(label[:, 0])
d_label = []
for i in range(len(l_train)):
    idx = label_mac.index(mh_list_train[i])
    d_label.append([mh_list_train[i], label_list[idx]])

df_label = pd.DataFrame(data=d_label, columns=['mac_hash', 'label'])

d_train = np.where(d_train < min_limit, 30, d_train)
# d_train = np.where(d_train < min_limit, 0, d_train)
d_train = np.where(d_train > max_limit, mean, d_train)

d = np.ndarray.flatten(d_train)
d_mean = np.mean(d)
d_std = np.std(d)

d_train = (d_train - d_mean) / d_std
df_train = pd.DataFrame(data=d_train)
df_train = pd.concat([df_label, df_train], axis=1)
df_train.to_csv('train_time.csv')

dict_train = {}

for i in range(len(l_train)):
    dict_train[i] = l_train[i][0] + d_train[i].tolist()
#     dict_train[i] = [l_train[i][0], l_location, d_train[i].tolist()]

jlb.dump(dict_train, 'train_feature_stay.pkl')
# df_train

['train_feature_stay.pkl']

In [43]:
df_train

,mac_hash,label,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0003ae8541d0e925fcee242287e2ad27,0,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
1,00078611037990f7f36b722f22595fe7,1,-0.391594,3.042811,-0.091334,-0.391594,-0.391594,-0.124696,-0.391594,-0.391594,-0.391594,3.042811,-0.391594,-0.391594,-0.391594,-0.391594
2,000ce31739b333d931813de403156844,3,-0.158058,-0.391594,-0.391594,3.042811,-0.391594,-0.391594,-0.391594,0.617614,3.042811,-0.391594,0.567571,3.042811,-0.391594,-0.391594
3,0027240b2aeb1f7f3c053c073cafb452,4,-0.391594,1.059664,-0.391594,3.042811,1.226475,3.042811,-0.191420,1.735250,0.250629,0.475825,-0.391594,-0.391594,3.042811,3.042811
4,0028a89964416dc9a8a66cc7fe25ef4e,4,-0.391594,3.042811,-0.391594,-0.391594,-0.391594,3.042811,-0.391594,-0.391594,1.251497,3.042811,-0.299848,3.042811,3.042811,-0.391594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6954,ffca215be2cb1505d931bd162da60104,4,-0.391594,3.042811,-0.391594,1.059664,3.042811,0.817788,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,0.400759,0.384078,-0.391594
6955,ffcaf0f29c6bede7b5cd9bc807c31715,1,-0.391594,3.042811,-0.391594,0.901193,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
6956,ffd0c4403c3c44dcbe4772821b032252,4,-0.391594,0.425781,1.359924,-0.174739,0.909534,0.617614,3.042811,-0.032950,0.517527,3.042811,0.309013,-0.383253,3.042811,-0.391594
6957,ffd7be58ee3c3fb47530ae7e4f251f88,0,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594


In [44]:
for i in range(len(l_train)):
    for j in range(1, len(l_train[i])-1):
        if l_train[i][j][1] == l_train[i][j+1][1]:
            diff = l_train[i][j+1][2] - l_train[i][j][2]
            if diff > max_limit:
                diff = max_limit
            elif diff < min_limit:
                diff = min_limit
        else:
            diff = min_limit
        l_train[i][j][2] = (diff - d_mean) / d_std
#         l_train[i][j][2] = diff
            
#     l_train[i][len(l_train[i])-1][2] = (min_limit - d_mean) / d_std
    l_train[i][len(l_train[i])-1][2] = 0.


for i in range(len(l_train)):
    num = len(l_train[i]) - 1
    k = (14 - num)
    for j in range(k):
        l_train[i].append([l_train[i][num][0], l_train[i][num][1], 0., 0., l_train[i][num][4]])
#         l_train[i].append([0.])

dict_train = {}
for i in range(len(l_train)):
    if len(l_train[i]) != 15:
        print(i, len(l_train[i]))
    dict_train[i] = l_train[i]

jlb.dump(dict_train, 'train_feature.pkl')

# print(dict_train)

['train_feature.pkl']

In [45]:
d_test = np.where(d_test < min_limit, 30, d_test)
# d_test = np.where(d_test < min_limit, 0, d_test)
d_test = np.where(d_test > max_limit, mean, d_test)
 
d_test = (d_test - d_mean) / d_std
df_test = pd.DataFrame(data=d_test)
test_mac_df = pd.DataFrame(data=mh_list_test, columns=['mac_hash'])
df_test = pd.concat([test_mac_df,df_test], axis=1)
df_test.to_csv('test_time.csv')

dict_test = {}

for i in range(len(l_test)):
    dict_test[i] = l_test[i][0] + d_test[i].tolist()
#     dict_test[i] = [l_test[i][0], l_location, d_test[i].tolist()]

jlb.dump(dict_test, 'test_feature_stay.pkl')
# df_test
# dict_test

['test_feature_stay.pkl']

In [46]:
df_test

,mac_hash,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,00c41469578fb947de736d3a03900772,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
1,0104474d9c1318b4f58a75fb605f39b2,-0.391594,0.359057,-0.391594,-0.141377,-0.391594,3.042811,-0.391594,3.042811,-0.141377,-0.391594,-0.391594,3.042811,-0.391594,-0.391594
2,01301359ac30dd5967c328f58282f13c,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
3,01911600eaa3baca34cf3e2f10ca353f,-0.391594,0.359057,-0.391594,-0.391594,3.042811,0.359057,-0.391594,-0.141377,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
4,01ca752c7508d63665a2f62725286034,-0.391594,0.359057,-0.391594,-0.391594,-0.391594,-0.141377,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,fed011e18897bfebe6b1d06288d9ed8d,-0.391594,-0.141377,-0.391594,-0.141377,3.042811,0.359057,-0.391594,3.042811,-0.391594,-0.391594,-0.391594,-0.141377,-0.391594,-0.141377
3425,fed9dd416420f3dfe6104b3ada5798e6,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
3426,feec5b8074464286c77c98096b8e5265,-0.391594,-0.391594,-0.391594,-0.391594,3.042811,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594,-0.391594
3427,ffb29a79ebd3034876cc20c3d553dda6,-0.391594,0.359057,3.042811,3.042811,3.042811,3.042811,-0.391594,-0.391594,3.042811,-0.391594,1.359924,3.042811,-0.391594,-0.391594


In [47]:
for i in range(len(l_test)):
    for j in range(1, len(l_test[i])-1):
        if l_test[i][j][1] == l_test[i][j+1][1]:
            diff = l_test[i][j+1][2] - l_test[i][j][2]
            if diff > max_limit:
                diff = max_limit
            elif diff < min_limit:
#                 diff = 0
                diff = min_limit
        else:
            diff = min_limit
        l_test[i][j][2] = (diff - d_mean) / d_std
#         l_test[i][j][2] = diff
            
#     l_test[i][len(l_test[i])-1][2] = (min_limit - d_mean) / d_std
    l_test[i][len(l_test[i])-1][2] = 0.


for i in range(len(l_test)):
    num = len(l_test[i]) - 1
    k = (14 - num)
    for j in range(k):
        l_test[i].append([l_test[i][num][0], l_test[i][num][1], 0., 0., l_test[i][num][4]])
#         l_test[i].append([0.])

dict_test = {}
for i in range(len(l_test)):
    if len(l_test[i]) != 15:
        print(i, len(l_test[i]))
    dict_test[i] = l_test[i]

jlb.dump(dict_test, 'test_feature.pkl')

# print(dict_test)

['test_feature.pkl']

In [48]:
dict_test

{0: [['00c41469578fb947de736d3a03900772'],
  [3.0, 0.0, 0.0, -1.3215960856502083, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0],
  [3.0, 0.0, 0.0, 0.0, 1.0]],
 1: [['0104474d9c1318b4f58a75fb605f39b2'],
  [10.0, 0.0, -0.39159397555251696, -0.7881954925843574, 5.0],
  [9.0, 0.0, -0.1413770911514097, -0.5115780740946178, 5.0],
  [11.0, 0.0, -0.39159397555251696, 0.028672855833660926, 5.0],
  [8.0, 0.0, 1.8770391096841887, 1.0258118067688302, 4.0],
  [12.0, 0.0, 1.8770391096841887, 1.9112997124565576, 6.0],
  [4.0, 0.0, -0.1413770911514097, -0.4973661476522824, 2.0],
  [7.0, 0.0, -0.39159397555251696, -0.026194645347427443, 3.0],
  [2.0, 0.0, 0.3590566776508048, 0.3091569409142925, 2.0],
